In [2]:
import pandas as pd
import numpy as np

# Load historical generation data (training target)
gen_df = pd.read_csv(r"C:\Users\tomas\Downloads\repsol_dataset\Generacion_fotovoltaica.csv", parse_dates=['FECHA'])
gen_df.rename(columns={'FECHA': 'Datetime', 'TOTAL_KWH_ENERGIA': 'GEN_KWH'}, inplace=True)

# Load actual solar energy used (for underutilization)
actual_used_df = pd.read_csv(r"C:\Users\tomas\Downloads\repsol_dataset\Consumo_fotovoltaica.csv", parse_dates=['FECHA'])
actual_used_df.rename(columns={'FECHA': 'Datetime', 'TOTAL_KWH_ENERGIA': 'ACTUAL_USED_KWH'}, inplace=True)

# Load meteorological data
df_raw = pd.read_excel(r"C:\Users\tomas\Downloads\repsol_dataset\Meteorologia.xlsx", header=None)

# Extract column names from the first row
columns = df_raw.iloc[0, 0].split(',')

# Split the rest of the data (starting from second row) by comma
meteo_df = df_raw.iloc[1:, 0].str.split(',', expand=True)

# Assign column names
meteo_df.columns = columns

meteo_df['FORECAST_TIMESTAMP'] = pd.to_datetime(meteo_df['FORECAST_TIMESTAMP'])

meteo_df.rename(columns={'FORECAST_TIMESTAMP': 'Datetime'}, inplace=True)

# Convert weather timestamps from UTC to Europe/Madrid
meteo_df['Datetime'] = meteo_df['Datetime'].dt.tz_convert('Europe/Madrid').dt.tz_localize(None)

In [3]:
# Select key features
weather_features = [
    'dswrfsurface_0',   # Downward shortwave radiation
    'SUNSDsurface_0',   # Sunshine duration
    'tccatmosphere_0',  # Total cloud cover
    '2theightAboveGround_2',
    '2rheightAboveGround_2',
    'dlwrfsurface_0',    # Air temp at 2m
]

# Merge generation with weather on timestamp
df = pd.merge(gen_df, meteo_df[['Datetime'] + weather_features], on='Datetime', how='inner')

# Add hour and day features
df['hour'] = df['Datetime'].dt.hour
df['dayofweek'] = df['Datetime'].dt.dayofweek
df

,Datetime,GEN_KWH,dswrfsurface_0,SUNSDsurface_0,tccatmosphere_0,2theightAboveGround_2,2rheightAboveGround_2,dlwrfsurface_0,hour,dayofweek
0,2023-07-24 13:00:00,129.98,567.66,18000.0,0.8,301.66785888671876,19.6,318.40769042968753,13,0
1,2023-07-24 13:00:00,129.98,570.04,18000.0,0.0,301.7678588867188,17.5,319.8076904296875,13,0
2,2023-07-24 13:00:00,129.98,572.5,18000.0,0.0,301.36785888671875,15.9,319.20769042968755,13,0
3,2023-07-24 13:00:00,129.98,566.1,18000.0,5.0,301.66785888671876,18.6,320.20769042968755,13,0
4,2023-07-24 13:00:00,129.98,571.02,18000.0,3.6,301.7678588867188,17.400000000000002,318.8076904296875,13,0
...,...,...,...,...,...,...,...,...,...,...
87358,2024-08-31 23:00:00,NaN,8.272,2553.0,100.0,298.63662109375,53.900000000000006,376.817041015625,23,5
87359,2024-08-31 23:00:00,NaN,7.152,2539.0,100.0,297.93662109375003,57.400000000000006,375.017041015625,23,5
87360,2024-08-31 23:00:00,NaN,8.704,2700.0,100.0,298.43662109375003,51.0,368.41704101562505,23,5
87361,2024-08-31 23:00:00,NaN,7.5040000000000004,2592.0,100.0,300.23662109375,45.6,380.717041015625,23,5


In [4]:
#check for null values 
df.isna().sum()

Datetime                     0
GEN_KWH                  10467
dswrfsurface_0               0
SUNSDsurface_0               0
tccatmosphere_0              0
2theightAboveGround_2        0
2rheightAboveGround_2        0
dlwrfsurface_0               0
hour                         0
dayofweek                    0
dtype: int64

In [5]:
#lets see the distribution of the null values per hour
df[df['GEN_KWH'].isna()]['hour'].value_counts().sort_index()


hour
0     414
1     414
2     414
3     414
4     414
5     414
6     414
7     441
8     468
9     504
10    450
11    450
12    459
13    459
14    459
15    459
16    450
17    432
18    423
19    423
20    423
21    423
22    423
23    423
Name: count, dtype: int64

In [6]:
import numpy as np

# Step 1: Fill night-time NaNs (hours 0-5 and 19-23) with 0
df.loc[df['hour'].isin([0, 1, 2, 3, 4, 5, 19, 20, 21, 22, 23]) & df['GEN_KWH'].isna(), 'GEN_KWH'] = 0

# Step 2: Fill day-time NaNs (hours 6-18) with that hour's mean
for h in range(6, 19):  # 6 to 18 inclusive
    hour_mask = (df['hour'] == h)
    hour_mean = df.loc[hour_mask & df['GEN_KWH'].notna(), 'GEN_KWH'].mean()
    df.loc[hour_mask & df['GEN_KWH'].isna(), 'GEN_KWH'] = hour_mean


In [7]:
print("Remaining NaNs:", df['GEN_KWH'].isna().sum())


Remaining NaNs: 0


In [8]:
# Loop through each of the selected weather-related columns to Convert the column values to numeric (float), coercing errors to NaN
# This ensures the column contains valid numerical values only
for col in ['dswrfsurface_0', 'SUNSDsurface_0', 'tccatmosphere_0', '2theightAboveGround_2', '2rheightAboveGround_2','dlwrfsurface_0']:
    df[col] = pd.to_numeric(df[col], errors='coerce')



In [9]:
# Import necessary modules for training, modeling, and evaluation
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error


# Define the input features (weather + time-based)
X = df[weather_features + ['hour', 'dayofweek']]

# Define the target variable: actual solar energy generated (in kWh)
y = df['GEN_KWH']


# Split the data into training and validation sets (80% train, 20% validation)
# random_state ensures reproducibility
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize the LightGBM regressor with specified hyperparameters
model = LGBMRegressor(
    n_estimators=200,     # Number of boosting rounds (trees)
    learning_rate=0.08,   # Step size shrinkage used in each boosting step
)


# Fit the model on the training data
model.fit(X_train, y_train)


# Predict on the validation set
val_preds = model.predict(X_val)

# Calculate and print the Mean Absolute Error (MAE) on validation predictions
print("Validation MAE:", mean_absolute_error(y_val, val_preds))


c:\Users\tomas\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\tomas\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\tomas\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tomas\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\tomas\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000935 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 69890, number of used features: 8
[LightGBM] [Info] Start training from score 25.358757
Validation MAE: 6.355279043940952


In [10]:
# Filter weather data for September 2024
sept_2024 = meteo_df[(meteo_df['Datetime'] >= '2024-09-01') & (meteo_df['Datetime'] < '2024-10-01')].copy()
sept_2024['hour'] = sept_2024['Datetime'].dt.hour
sept_2024['dayofweek'] = sept_2024['Datetime'].dt.dayofweek


# Some weather data may contain non-numeric values (e.g. strings, missing values).
# We convert them to numeric, coercing invalid values to NaN (to avoid errors).
bad_cols = ['dswrfsurface_0', 'SUNSDsurface_0', 'tccatmosphere_0', '2theightAboveGround_2', '2rheightAboveGround_2','dlwrfsurface_0']
for col in bad_cols:
    sept_2024[col] = pd.to_numeric(sept_2024[col], errors='coerce')

# Predict
X_pred = sept_2024[weather_features + ['hour', 'dayofweek']]
sept_2024['KWH_ENERGIA'] = model.predict(X_pred)

In [11]:
# Keep only the first occurrence of each datetime
sept_2024 = sept_2024.drop_duplicates(subset='Datetime', keep='first')

In [12]:
sept_2024[['Datetime', 'KWH_ENERGIA']].to_csv("Solar_Prediction.csv", index=False)